In [8]:
import pandas as pd
from transformers import BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import torch
from torch import nn
from sklearn.metrics import accuracy_score, auc
from sklearn.model_selection import train_test_split
from transformers import BertConfig, BertForPreTraining, BertModel
import json
def get_result(model, data_loader,  device='cuda'):
    model.eval()
    model = model.to(device)
    losses = []
    labels_list = []
    preds_list = []
    data_dict = {}
    for data_labels in tqdm(data_loader):
        data = data_labels[0]
        
        next_sentence_label = data_labels[1].to(device).long()
        data_dict['input_ids'] = data['input_ids'].to(device).long()
        data_dict['token_type_ids'] = data['token_type_ids'].to(device).long()
        data_dict['attention_mask'] =  data['attention_mask'].to(device).long()
        outputs = model(data_dict)
        preds = outputs[:,0,5:7].cpu().detach().numpy()
        preds = preds[:, 1] / (preds.sum(axis=1) + 1e-8)
        preds_list.append(preds)
    results = np.concatenate(preds_list)
    return results
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
##查看训练集和测试集中字
train = pd.read_csv('./data/gaiic_track3_round1_train_20210228.tsv',sep='\t', names=['text_a', 'text_b', 'label'])
test = pd.read_csv('data/gaiic_track3_round1_testA_20210228.tsv',sep='\t', names=['text_a', 'text_b', 'label'])
test['label'] = 2
from collections import defaultdict
def get_dict(data):
    words_dict = defaultdict(int)
    for i in tqdm(range(data.shape[0])):
        text = data.text_a.iloc[i].split() + data.text_b.iloc[i].split()
        for c in text:
            words_dict[c] += 1
    return words_dict
word_dict = get_dict(train.append(test))
min_count = 5
word_dict =  {i: j for i, j in word_dict.items() if j >= min_count}
word_dict = dict(sorted(word_dict.items(), key=lambda s: -s[1]))
word_dict = list(word_dict.keys())
special_tokens = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]", "sim", 'sim_no', 'unlabeled']
WORDS = special_tokens + word_dict
pd.Series(WORDS).to_csv('Bert-vocab.txt', header=False,index=0)
vocab = pd.read_csv('Bert-vocab.txt', names=['word'])
vocab_dict = {}
for key, value in vocab.word.to_dict().items():
    vocab_dict[value] = key

100%|██████████| 125000/125000 [00:03<00:00, 35393.14it/s]


In [12]:
class PretrainedBERT(nn.Module):
    def __init__(self, embeding_size=6933, embedding_dim=768, maxlen=64, checkpoint_path='embeddingBert.pth',keep_tokens=None):
        super(PretrainedBERT, self).__init__()
#         self.bert = BertModel(BertConfig(embeding_size))
        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.lr = nn.Linear(in_features=embedding_dim, out_features=768)
        self.layer_norm = nn.LayerNorm((maxlen, embedding_dim))
        self.lr1 = nn.Linear(in_features=768, out_features=embeding_size)
        ##使用bert的预训练embedding作为初始化embedding
        if keep_tokens is not None:
            self.embedding = nn.Embedding(embeding_size, embedding_dim)
            weight = torch.load('embeddingBert.pth')
            weight = nn.Parameter(weight['weight'][keep_tokens])
            self.embedding.weight = weight
            self.bert.embeddings.word_embeddings = self.embedding
    def forward(self, x):
        x = self.bert(**x)
        x = self.lr(x['last_hidden_state'])
        x = self.layer_norm(x)
        x = self.lr1(x)
        return x
class OPPODataset(Dataset):
    def __init__(self, data, word_dict,seq_length=50):
        '''
        data:dataFrame()
        '''
        self.data = data
        self.vocab = word_dict
        self.seq_len = seq_length
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, index):
        '''
        transformers 中可以用BertTokenizer实现下面的方法但是有一些不灵活
        transoformers格式
        0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence |
        '''
        text_a, text_b, label = self.data.iloc[index].values
        text_a = self.get_sentence(text_a)
        text_b = self.get_sentence(text_b)
        text_a = [self.vocab['[CLS]']] + text_a + [self.vocab['[SEP]']]
        text_b = text_b + [self.vocab['[SEP]']]

        token_type_ids = ([0 for _ in range(len(text_a))] + [1 for _ in range(len(text_b))])[:self.seq_len]
        text = (text_a + text_b)[:self.seq_len]

        padding = [self.vocab['[PAD]'] for _ in range(self.seq_len - len(text))]
        attention_mask = len(text) * [1]
        
        text.extend(padding), token_type_ids.extend(padding), attention_mask.extend(padding)
        attention_mask = np.array(attention_mask)
        text = np.array(text)
        token_type_ids = np.array(token_type_ids)
        return {
                'input_ids': text,
                'token_type_ids': token_type_ids,
                'attention_mask': attention_mask,
               }, self.data.label.iloc[index]
    def get_sentence(self, sentence):
        tokens = sentence.split()
        for i in range(len(tokens)):
            tokens[i] = self.vocab.get(tokens[i], self.vocab['[UNK]'])
        return tokens

In [13]:
with open('bert-base-chinese-1/vocab.txt', 'r') as f:
    print(f)
    lines = f.read()
    tokens = lines.split('\n')
token_dict = dict(zip(tokens, range(len(tokens))))
counts = json.load(open('counts.json'))
del counts['[CLS]']
del counts['[SEP]']
freqs = [
    counts.get(i, 0) for i, j in sorted(token_dict.items(), key=lambda s: s[1])
]
keep_tokens = list(np.argsort(freqs)[::-1])
keep_tokens = [0, 100, 101, 102, 103, 6, 7, 8] + keep_tokens[:len(vocab_dict)]

<_io.TextIOWrapper name='bert-base-chinese-1/vocab.txt' mode='r' encoding='UTF-8'>


In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = PretrainedBERT(embeding_size=len(vocab_dict), keep_tokens=keep_tokens)
model.load_state_dict(torch.load('pretrainBERT/preTrainModel0.923.pth',  map_location=device))
model = model.to(device)

test_dataset = OPPODataset(test, vocab_dict, seq_length=64)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [16]:
result = get_result(model, test_loader)

100%|██████████| 391/391 [00:45<00:00,  8.58it/s]


In [17]:
test['label'] = result
test['label'].to_csv('sub38.tsv', sep='\t', index=0, header=False)